In [2]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

engine = sa.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
    .format(host=os.getenv('host'), db=os.getenv('db'), user=os.getenv('uname'), pw=os.getenv('password')))

## Make Prime table in DB

In [3]:
df = pd.read_csv("prime.csv")
df['DATE'] = pd.to_datetime(df['DATE'])

df.to_sql(name="prime_rates", con= engine, if_exists='replace', index=False)

query_mod_date_col = """ALTER TABLE `sba`.`prime_rates` 
CHANGE COLUMN `DATE` `DATE` DATE NOT NULL ,
ADD UNIQUE INDEX `DATE_UNIQUE` (`DATE` ASC);
;"""

engine.execute(query_mod_date_col)

# query to add date and year columns for easier joins
add_mth_year_queries = []
add_mth_year_queries.append(sa.text("ALTER TABLE prime_rates ADD COLUMN p_MONTH int NULL AFTER date ;"))
add_mth_year_queries.append(sa.text("ALTER TABLE prime_rates ADD COLUMN p_YEAR int NULL AFTER date ;"))
add_mth_year_queries.append(sa.text("UPDATE prime_rates SET prime_rates.p_MONTH = MONTH(prime_rates.`DATE`);"))
add_mth_year_queries.append(sa.text("UPDATE prime_rates SET prime_rates.p_YEAR = YEAR(prime_rates.`DATE`);"))

                # INSERT INTO prime_rates(month) FROM prime_rates;
for q in add_mth_year_queries:
    engine.execute(q)

prime_df = pd.read_sql("SELECT * FROM sba.prime_rates",engine)

## Read in loan data, format dates

In [4]:
loan_df = pd.read_sql('SELECT * FROM clean_version', con=engine)
date_cols = [c for c in loan_df.columns if 'Dt' in c]
for c in date_cols:
    loan_df[c] = pd.to_datetime(loan_df[c])
loan_df['Note_Yr'] = loan_df['NoteDt'].dt.year
loan_df['Note_Mth'] = loan_df['NoteDt'].dt.month
first_cols = ['GP','Note_Yr','Note_Mth'] 
formatted_order = first_cols + [c for c in loan_df.columns if c not in first_cols]
loan_df = loan_df[formatted_order] 
loan_df.head()

,GP,Note_Yr,Note_Mth,NoteDt,LoanAmt,MaturityDt,MaturityMthsQty,PrepayDt,PrepayMthsQty,DefaultDt,DefaultMthsQty,MatBucket
0,1502649,2000,7,2000-07-31,20600.00,2007-07-31,84,2008-10-08,NaN,2006-03-06,67.0,0-8
1,1503079,2001,1,2001-01-31,14288.55,2008-01-31,84,2010-11-30,NaN,2004-03-02,37.0,0-8
2,2019667,2001,2,2001-02-28,3600.00,2003-02-28,24,2002-01-30,11.0,NaT,NaN,0-8
3,2022516,2001,2,2001-02-28,60000.00,2011-02-27,119,2011-09-13,NaN,2010-01-12,106.0,8-11
4,2039790,2001,7,2001-07-31,20000.02,2006-07-31,60,2005-05-12,45.0,2003-12-31,29.0,0-8


In [5]:
test_df = loan_df[loan_df['MatBucket']=='21+']

In [6]:
gp_checks = test_df[test_df['PrepayMthsQty']==0]['GP'].to_list()

for gp in gp_checks: 
    failed_gps = []
    gp_query = f"SELECT EffectiveDt, TransactionBalanceAmt from FIN_FINAL where ObservationNmb = {gp} AND GeneralLedgerCd in (6031,1510) ORDER BY EffectiveDt;"
    test_gp = pd.read_sql(gp_query, engine)
    # case_1 --> where there are two instances of the balance being zero:
    zero_counts= test_gp[test_gp['TransactionBalanceAmt']==0].shape[0]
    if zero_counts > 1:
        failed_gps.append(gp)
    else:
    # case_2 --> where there is one zero but the loan is still outstanding
        if test_gp.tail(1)['TransactionBalanceAmt'].values[-1]>0:
        # action for case_2 --> go to table and change the PrepayDt column and PrepayMthsQty Column to Null
            test_df.loc[test_df['GP']==gp,'PrepayMthsQty'] = np.nan
            test_df.loc[test_df['GP']==gp,'PrepayDt'] = pd.NaT



In [7]:
test_df[test_df['PrepayMthsQty']==0]

,GP,Note_Yr,Note_Mth,NoteDt,LoanAmt,MaturityDt,MaturityMthsQty,PrepayDt,PrepayMthsQty,DefaultDt,DefaultMthsQty,MatBucket
8203,4237360,2006,3,2006-03-31,300000.0,2031-03-31,300,2006-04-30,0.0,NaT,NaN,21+
89648,10711291,2021,2,2021-02-28,NaN,2046-01-19,298,2021-03-03,0.0,NaT,NaN,21+
192987,2347229,2003,1,2003-01-31,573300.0,2028-01-31,300,2003-02-28,0.0,NaT,NaN,21+
346594,2016954,2001,1,2001-01-31,72300.0,2026-01-31,300,2001-02-28,0.0,NaT,NaN,21+
467821,17983847,2021,9,2021-09-28,350000.0,2046-09-28,300,2021-10-01,0.0,NaT,NaN,21+
736215,4987580,2017,5,2017-05-31,119700.0,2042-04-26,298,2017-06-30,0.0,NaT,NaN,21+


In [8]:
def coalesce_least(row):
    a = row['PrepayMthsQty']
    b = row['DefaultMthsQty']
    if pd.isnull(a):
        return b
    elif pd.isnull(b):
        return a
    else:
        return min(a, b)
    
test_df['PP_Qty'] = test_df[['PrepayMthsQty','DefaultMthsQty']].apply(coalesce_least, axis=1)

In [9]:
# group_df = test_df[['Note_Yr','Note_Mth','PP_Qty']].groupby(['Note_Yr','Note_Mth'])['PP_Qty'].agg(total_count= 'count', PP_within_FOUR= lambda x: x[x<=48].count()/len(x), \
#                                                     PP_within_SEVEN= lambda x: x[x<=86].count()/len(x) )

# group_df[['PP_within_FOUR']].plot()

In [10]:
# Group by year and month of origination
df_grouped = test_df.groupby(['Note_Yr', 'Note_Mth'])


def balance_rundown(grp):
    balance_dict = {}
    start_balance = grp['LoanAmt'].sum()
    balance_dict[0] = start_balance
    months_to_track = list(range(0, 121, 6))
    # Loop through each month and calculate balance rundown
    for month in months_to_track[1:]:
        # Remove loans with PP_Qty less than or equal to x months
        balance =round(grp.loc[(grp['PP_Qty'] > month) | (grp['PP_Qty'].isna()), 'LoanAmt'].sum(),2)
        # Add remaining balance to dictionary
        balance_dict[month] = balance
    
    return balance_dict


def loan_count(grp, date_str):
    loan_count_dict = {}
    n_loans = grp['GP'].count()
    loan_count_dict[0] = n_loans
    months_to_track = list(range(0, 252, 1))
    curr_time = dt.datetime.now()
    for month in months_to_track[1:]:
        test_date = pd.to_datetime(date_str) + np.timedelta64(month,'M')
        # conidition == FALSE if the test_date is LESS than the curr_date
        condition = test_date > curr_time
        if not condition:
        # Filter loans with PP_Qty greater than month or missing values
            outstanding_loans = grp.loc[(grp['PP_Qty'] > month) | (grp['PP_Qty'].isna())]
            # Count the number of outstanding loans
            n_outstanding_loans = outstanding_loans['GP'].count()
            # Add the number of outstanding loans to the dictionary
            loan_count_dict[month] = n_outstanding_loans
    return loan_count_dict

master_dict = {}
for name, grp in df_grouped:
    x = name
    yr = [y for y in x][0]
    mth = [y for y in x][1]

    date_str = f"{yr}-{mth}"
    master_dict[name] = loan_count(grp, date_str)
    

In [11]:
df_sample = pd.DataFrame(master_dict)
my_range = list(range(0,252,1))

# long way -- probs a way to vectorize this

data = {}
for idx, row in df_sample.transpose().iterrows():
    cohort_name = f"{idx[0]}-{idx[1]}"
    data_row = (row.to_list())
    survivorship = [round(100*(1-(data_row[0]-d)/data_row[0]), 3) for d in data_row]
    data[cohort_name] = survivorship
    


In [12]:
# pd.DataFrame(data).plot(figsize=(12,10), grid=True)
import plotly.express as px
surv = pd.DataFrame(data)

fig = px.line(data_frame=surv, line_shape='linear')
fig.show()
# surv.plot(figsize=(15,10), grid=True)

In [146]:
pd.DataFrame(master_dict)

In [104]:
prime_df.head()

,DATE,p_YEAR,p_MONTH,PRIME
0,1955-08-04,1955,8,3.25
1,1955-10-14,1955,10,3.50
2,1956-04-13,1956,4,3.75
3,1956-08-21,1956,8,4.00
4,1957-08-06,1957,8,4.50
